In [1]:
#finds site map of entire domain
from usp.tree import sitemap_tree_for_homepage

#for saving and loading objects
import pickle

#for accessing url html
import requests
import urllib.request, urllib.parse, urllib.error
from urllib.request import Request, urlopen

#for html parsing
from bs4 import BeautifulSoup

#anxillary functions/methods
import numpy as np
import pandas as pd
import time
import random
import datetime
from dateutil.tz import tzutc, tzoffset
from tqdm.notebook import tqdm



# Sitemap

In [ ]:
tree = sitemap_tree_for_homepage('https://hongkongfp.com/');

In [ ]:
urls = []
dates = []
for page in tree.all_pages():
    urls.append(page.url)
    dates.append(page.last_modified)
    
#transpose
zippedList =list(zip(urls,dates))

#format into dataframe
df = pd.DataFrame(zippedList, columns = ['urls' ,'date'])

#drop nan for date published
df = df.dropna()
        
pickle.dump(df, open("../Pickles/HKFP_All.p", "wb" ) )


# Data Scrape

In [7]:
#LOAD DATASET OF ALL SCRAPED URLS FROM SCMP
with open("../Pickles/HKFP_All.p", 'rb') as f:
    df = pickle.load(f)
          
#reset df index
df.reset_index(drop = True)

#find articles with "Articles" to exclude photos/video pages
articles_only_urls = []
for j in df.urls:
   # if '2018' in i or '2019' in i or '2020' in i:
    if any(i.isdigit() for i in j):
        articles_only_urls.append(j)
        
author = []
title = []
date = []
summary = []
link = []
broken = []

articles_only_urls = list(dict.fromkeys(articles_only_urls))

In [15]:
#using BS4 to request each video html data
for i in tqdm(articles_only_urls):
    try:
        
        time.sleep(0.5) 
        req = Request(str(i), headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'lxml')

        by = soup.find("a", {"class": "url fn n"})
        t = soup.find("h1", {"class": "entry-title"})
        d = soup.find("time", {"class": "entry-date published"})


        if by and t and d:
            author.append(by.text)
            title.append(t.text.strip())
            date.append(d.text)
            link.append(i)

            if len(link)% 1000 == 0:
                print("Dataset contains " + str(len(link)) + " entries.")
    except:
        print("URL number " + str(articles_only_urls.index(i)) + " is broken: " + i )
        broken.append(i)
        pass

Dataset contains 1000 entries.
Dataset contains 2000 entries.
Dataset contains 3000 entries.
Dataset contains 4000 entries.
Dataset contains 5000 entries.
Dataset contains 6000 entries.
Dataset contains 7000 entries.
Dataset contains 8000 entries.
Dataset contains 9000 entries.
Dataset contains 10000 entries.
Dataset contains 11000 entries.
Dataset contains 12000 entries.
Dataset contains 13000 entries.
Dataset contains 14000 entries.
Dataset contains 15000 entries.
Dataset contains 16000 entries.
Dataset contains 17000 entries.
URL number 17827 is broken: https://hongkongfp.com/newsletter/hkfp-dim-sum-28-08-interview-with-top-pollster-robert-chung-digital-security-guide/
URL number 17828 is broken: https://hongkongfp.com/newsletter/hkfp-dim-sum-04-09-the-hong-kong-artists-resisting-censorship/
URL number 17829 is broken: https://hongkongfp.com/newsletter/hkfp-dim-sum-11-09-exclusive-interviews-with-jimmy-lai-jeffrey-wasserstrom-baggio-leung/
URL number 17830 is broken: https://hongkon

In [16]:
df = pd.DataFrame(list(zip(title, author, date, link)), 
               columns =['Title', 'Author', 'Date', 'Link']) 
    

with open('../Pickles/HKFP_News_Full.pkl', 'wb') as f:
    pickle.dump(df, f)

,Title,Author,Date,Link
0,In Pictures: Hundreds protest against Tuen Mun...,Rachel Wong,"15:09, 24 February 2020",https://hongkongfp.com/2020/02/24/pictures-hun...
1,In Pictures: Hundreds protest against Tuen Mun...,Rachel Wong,"15:09, 24 February 2020",https://hongkongfp.com/2020/02/24/pictures-hun...
2,Traces of coronavirus detected in Hong Kong pa...,Kelly Ho,"13:00, 28 February 2020",https://hongkongfp.com/2020/02/28/traces-coron...
3,China censors report about how authorities hid...,Global Voices,"09:00, 7 March 2020",https://hongkongfp.com/2020/03/07/china-censor...
4,Community film screenings across Hong Kong mar...,Rachel Wong,"20:22, 13 March 2020",https://hongkongfp.com/2020/03/13/community-fi...
...,...,...,...,...
17808,In Pictures: The art of herping in Hong Kong,Chermaine Lee,"14:00, 7 November 2020",https://hongkongfp.com/2020/11/07/in-pictures-...
17809,Hong Kong’s arrest of 8 democrats is straight ...,Stephen Vines,"14:21, 7 November 2020",https://hongkongfp.com/2020/11/07/why-hong-kon...
17810,Chief Exec. Lam disputes claim journalist’s ar...,Rhoda Kwan,"13:55, 7 November 2020",https://hongkongfp.com/2020/11/07/chief-exec-l...
17811,"Democrat Joe Biden wins US election, ending Do...",AFP,"00:30, 8 November 2020",https://hongkongfp.com/2020/11/08/breaking-dem...


# Data Cleaning

In [19]:
df1 = df[df.Link.str.contains(".com/20")]
df1

,Title,Author,Date,Link
0,In Pictures: Hundreds protest against Tuen Mun...,Rachel Wong,"15:09, 24 February 2020",https://hongkongfp.com/2020/02/24/pictures-hun...
1,In Pictures: Hundreds protest against Tuen Mun...,Rachel Wong,"15:09, 24 February 2020",https://hongkongfp.com/2020/02/24/pictures-hun...
2,Traces of coronavirus detected in Hong Kong pa...,Kelly Ho,"13:00, 28 February 2020",https://hongkongfp.com/2020/02/28/traces-coron...
3,China censors report about how authorities hid...,Global Voices,"09:00, 7 March 2020",https://hongkongfp.com/2020/03/07/china-censor...
4,Community film screenings across Hong Kong mar...,Rachel Wong,"20:22, 13 March 2020",https://hongkongfp.com/2020/03/13/community-fi...
...,...,...,...,...
17808,In Pictures: The art of herping in Hong Kong,Chermaine Lee,"14:00, 7 November 2020",https://hongkongfp.com/2020/11/07/in-pictures-...
17809,Hong Kong’s arrest of 8 democrats is straight ...,Stephen Vines,"14:21, 7 November 2020",https://hongkongfp.com/2020/11/07/why-hong-kon...
17810,Chief Exec. Lam disputes claim journalist’s ar...,Rhoda Kwan,"13:55, 7 November 2020",https://hongkongfp.com/2020/11/07/chief-exec-l...
17811,"Democrat Joe Biden wins US election, ending Do...",AFP,"00:30, 8 November 2020",https://hongkongfp.com/2020/11/08/breaking-dem...


In [20]:
clean_list = ["Associated Press", 
"AP", 
"AFP",
"Agence France-Presse", 
"Reuters",  
"The Guardian", 
"Guardian News Service",
"Agencies", 
"Bloomberg", 
"GUARDIAN",
"The Washington Post",
"WASHPOST",
"@scmp.com",
"McClatchy",
"MCT",
"MCY",
"NYT",
"Straits Times",
"POLITICO",
"#Tech in Asia",
"Xinhua",
"WIRE",
"Science magazine",
"USA TODAY",
"Tribune News Service",
".com",]

In [21]:
for i in clean_list:
    df1 = df1[~df1.Author.str.contains(i)]

In [23]:
with open('../Pickles/HKFP_News_Full_Clean.pkl', 'wb') as f:
    pickle.dump(df1, f)